In [25]:
pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


In [1]:
from datasets import Dataset


In [2]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [3]:
import torch
import transformers
torch.__version__, transformers.__version__

('2.1.0+cu118', '4.35.2')

In [4]:
torch.cuda.is_available()

True

In [5]:
df = pd.read_parquet('data.parquet')


In [6]:
df.head()

,input,output,instruction
0,"Teollisuuden Voima Oyj , the Finnish utility k...",neutral,What is the sentiment of this news? Please cho...
1,Sanofi poaches AstraZeneca scientist as new re...,neutral,What is the sentiment of this news? Please cho...
2,Starbucks says the workers violated safety pol...,moderately negative,What is the sentiment of this news? Please cho...
3,$brcm raises revenue forecast,positive,What is the sentiment of this tweet? Please ch...
4,Google parent Alphabet Inc. reported revenue a...,moderately negative,What is the sentiment of this news? Please cho...


In [7]:
df = df.rename(columns={df.columns[0]: 'text', df.columns[1]: 'label'})
df.head()

,text,label,instruction
0,"Teollisuuden Voima Oyj , the Finnish utility k...",neutral,What is the sentiment of this news? Please cho...
1,Sanofi poaches AstraZeneca scientist as new re...,neutral,What is the sentiment of this news? Please cho...
2,Starbucks says the workers violated safety pol...,moderately negative,What is the sentiment of this news? Please cho...
3,$brcm raises revenue forecast,positive,What is the sentiment of this tweet? Please ch...
4,Google parent Alphabet Inc. reported revenue a...,moderately negative,What is the sentiment of this news? Please cho...


In [8]:
df['label'].unique()

array(['neutral', 'moderately negative', 'positive', 'mildly positive',
       'negative', 'moderately positive', 'mildly negative',
       'strong positive', 'strong negative'], dtype=object)

In [9]:
df['label'] = df['label'].replace({'strong negative':0,'negative':1,'moderately negative':2,'mildly negative':3,'neutral':4, 'mildly positive':5,
                                   'moderately positive':6, 'positive':7,'strong positive':8})

In [10]:
df['label'].unique()

array([4, 2, 7, 5, 1, 6, 3, 8, 0])

In [11]:
df_train, df_test, = train_test_split(df, stratify=df['label'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(62184, 3) (7678, 3) (6910, 3)


In [12]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=9)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/62184 [00:00<?, ? examples/s]

Map:   0%|          | 0/6910 [00:00<?, ? examples/s]

Map:   0%|          | 0/7678 [00:00<?, ? examples/s]

In [32]:
pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.2 MB/s eta 0:00:00


In [35]:
pip install accelerate -U

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,            # evaluation dataset
        compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.430000,0.342935,0.877424
2,0.277200,0.326738,0.885239
3,0.207700,0.326119,0.896237
4,0.148300,0.370077,0.892475
5,0.109700,0.405993,0.891172


TrainOutput(global_step=9720, training_loss=0.254559126606694, metrics={'train_runtime': 6232.2998, 'train_samples_per_second': 49.888, 'train_steps_per_second': 1.56, 'total_flos': 2.045290772118528e+16, 'train_loss': 0.254559126606694, 'epoch': 5.0})

In [15]:
model.eval()
trainer.predict(dataset_test).metrics

{'test_loss': 0.33568066358566284,
 'test_accuracy': 0.8939828080229226,
 'test_runtime': 50.8917,
 'test_samples_per_second': 150.869,
 'test_steps_per_second': 4.716}

In [16]:
trainer.save_model('finbert-sentiment/')

In [17]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [18]:
!cp -r /content/finbert-sentiment /content/drive/MyDrive/ravenriai_sentiment/



In [20]:
!cp -r /content/temp /content/drive/MyDrive/ravenriai_sentiment/
